# Installing Dependencies

In [1]:
! pip install -U keras-nlp
! pip install -U keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 9.6 MB/s eta 0:00:00:00:01
  Attempting uninstall: keras-nlp
    Found existing installation: keras-nlp 0.7.0
    Uninstalling keras-nlp-0.7.0:
      Successfully uninstalled keras-nlp-0.7.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.


# Importing Dependencies

In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras_nlp
import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import time

2024-03-06 17:43:31.321908: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 17:43:31.322006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 17:43:31.450494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Loading Gemma

In [3]:
gemma = keras_nlp.models.GemmaCausalLM.from_preset('gemma_instruct_2b_en')

Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [15]:
gemma.generate('Tell me something about yourself',max_length=200)

I0000 00:00:1709747245.381921      34 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


'Tell me something about yourself and what you do.\n\nI am a large language model (LLM) and I am trained by Google. I am a conversational AI that can understand and generate human-like text, and I am able to perform a wide range of tasks, including translation, summarization, and question answering.\n\nI am still under development, but I am learning new things every day. I am also constantly being improved by Google and its partners.'

In [ ]:
keras.mixed_precision.set_global_policy("mixed_float16")

# Creating Pipeline and Processing the Dataset

In [4]:
import pandas as pd

In [5]:
train_data=[]
for file in os.listdir('/kaggle/input/reddit-top-1000'):
    data=pd.read_csv('/kaggle/input/reddit-top-1000/'+file)
    for d in data['title']:
        train_data.append(d)

In [6]:
train_df=pd.DataFrame(columns=["text"])

In [7]:
train_df['text']=train_data

In [8]:
train_df

,text
0,.gif I painted after seeing Django Unchained. ...
1,Adorable tribute to 9 of Tom Hanks' most famou...
2,I'm Samuel L Jackson and I'll record a video o...
3,The last video store in my area closed down a ...
4,"I once owned a DeLorean, so I took it to the B..."
...,...
17994,Quote by Plato.
17995,"One year of working hard. If I can do it, I kn..."
17996,I found this oddly motivating. [X-Post /r/funny]
17997,even super villains give good motivation somet...


In [9]:
train_df.dropna(inplace=True)

# Reducing the size of rows to 50 for memory efficiency

In [10]:
train_df=train_df[:50:]
train_df

,text
0,.gif I painted after seeing Django Unchained. ...
1,Adorable tribute to 9 of Tom Hanks' most famou...
2,I'm Samuel L Jackson and I'll record a video o...
3,The last video store in my area closed down a ...
4,"I once owned a DeLorean, so I took it to the B..."
5,The Green Dragon in Hobbiton. A real and opera...
6,Here's a feature that Rotten Tomatoes needs.
7,"All the Bonds, down the barrel"
8,"Charlie Chaplin in front of New York crowd, 1918"
9,The Chicago Tribune's cartoon tribute to Roger...


# Enabling LoRA in Gemma 

In [11]:
gemma.backbone.enable_lora(rank=4)
gemma.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

# With LoRA, we have reduced the trainable params to 1.3M from 2.5B

In [12]:
%%time

gemma.preprocessor.sequence_length = 128

optimizer=tf.keras.optimizers.AdamW(
        learning_rate=5e-5,
        weight_decay=0.01)

optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=['accuracy']
)

CPU times: user 10.4 ms, sys: 1.18 ms, total: 11.6 ms
Wall time: 9.3 ms


In [13]:
training_data=[d for d in train_df['text'].dropna()]

In [14]:
training_data

['.gif I painted after seeing Django Unchained. Such a good movie.',
 "Adorable tribute to 9 of Tom Hanks' most famous performances",
 "I'm Samuel L Jackson and I'll record a video of me saying any 300 word monologue you upvote the most by the end of tomorrow(Thursday night PST)",
 'The last video store in my area closed down a few days ago. I received this from them.',
 'I once owned a DeLorean, so I took it to the BTTF houses. [x-post from /r/BacktotheFuture]',
 'The Green Dragon in Hobbiton. A real and operating pub.',
 "Here's a feature that Rotten Tomatoes needs.",
 'All the Bonds, down the barrel ',
 'Charlie Chaplin in front of New York crowd, 1918',
 "The Chicago Tribune's cartoon tribute to Roger Ebert :'(",
 "Pixar story artist Josh Cooley created a book of movie stills as if it were a children's storybook.  ",
 'This is how they created moonlight for DJANGO UNCHAINED',
 'Goldeneye (1995): Derek Meddings puts the finishing touches on his Siberian landscape',
 'A 1927 Paramoun

# Fitting the Model

In [16]:
%%time
gemma.fit(training_data,epochs=1,batch_size=1)

W0000 00:00:1709747304.770095     107 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


50/50 ━━━━━━━━━━━━━━━━━━━━ 52s 206ms/step - accuracy: 0.2656 - loss: 1.2100
CPU times: user 50.4 s, sys: 1.42 s, total: 51.9 s
Wall time: 52.2 s


# Testing the trained Gemma model

In [17]:
gemma.generate("How are you doing?",max_length=200)

'How are you doing?\n\nI am doing well, thank you for asking. I am functioning normally and ready to assist you with your requests. Is there anything I can help you with today?'